In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
%cd drive/Shareddrives/CS704_Project/llm_based_modeling/
!ls

[Errno 2] No such file or directory: 'drive/Shareddrives/CS704_Project/llm_based_modeling/'
/content/drive/Shareddrives/CS704_Project/llm_based_modeling
predicted_truthfulness_final_2_class.csv  test_data.csv
predict_truthfulness.ipynb		  top_5_news_summarized_test_data.csv
processed_data_3_majority.csv		  topic_generation.ipynb
processed_data_5_majority.csv		  topic_modelled_test_data.csv
qllama2_topic_modelling.ipynb


In [3]:
!pip install --upgrade pip
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor
!pip install openai==0.28
!pip install requests
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [4]:
from collections import defaultdict
from datetime import datetime
import pandas as pd
import json
import numpy as np
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from google.colab import userdata
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [5]:
openai.api_key = userdata.get('OPENAI_API_KEY')

In [5]:
test_data = pd.read_csv("topic_modelled_test_data.csv")
print(test_data.shape)
test_data.head()

(104, 4)


,statement,cleaned_tweet,Truthfulness,topics
0,Statistics show thatmore people of color don't...,Dan PatrickIf theyre worried about people of c...,False,"Dan Patrick, drive-in voting, people of color,..."
1,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re..."
2,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon"
3,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ..."
4,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ..."


In [15]:
topic = test_data['topics'][100]
topic_list = topic.split(',')
topic_list = [topic.strip() for topic in topic_list]
print(topic_list)
print("https://www.politifact.com/search/?q=" +'%2C+'.join(topic_list))

['Biden', "Trump's policies", 'Obama-Biden kids in cages policy', 'human trafficking', 'cartels']
https://www.politifact.com/search/?q=Biden%2C+Trump's policies%2C+Obama-Biden kids in cages policy%2C+human trafficking%2C+cartels


# Searching Internet for news related to topics

In [48]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_top_5_news_content(topics):
  def get_politifact_articles(keywords, num_articles=5):
      base_url = "https://www.politifact.com/"
      search_url = f"{base_url}search/?q={'%2C+'.join(keywords)}"
      response = requests.get(search_url)

      if response.status_code != 200:
          # print("Failed to retrieve the page.")
          return None

      soup = BeautifulSoup(response.text, "html.parser")
      articles = []

      # Find articles that contain all keywords
      # print("length")
      # print(len(soup.find_all("div", class_="c-textgroup__title")))
      for link in soup.find_all("div", class_="c-textgroup__title"):
          article_url = urljoin(base_url, link.find("a", href=True)["href"])
          article_response = requests.get(article_url)
          #print(article_url)
          if article_response.status_code != 200:
              # print(f"Failed to retrieve article: {article_url}")
              continue

          article_soup = BeautifulSoup(article_response.text, "html.parser")

          # Check if the article contains all keywords
          article_text = article_soup.find("article", class_="m-textblock")
          #if article_text and all(keyword.lower() in article_text.get_text().lower() for keyword in keywords):
          articles.append(article_text.get_text())
          # print(len(articles))
          if len(articles) == num_articles:
              break

      return articles

  topic_list = topics.split(',')
  topic_list = [topic.strip() for topic in topic_list]
  print(topic_list)

  search_keywords = topic_list

  # Specify the number of articles you want to retrieve
  num_articles_to_extract = 5

  result = get_politifact_articles(search_keywords, num_articles_to_extract)

  cnt = 1
  final_result = ''
  for res in result:
    # print(res)
    upto_fact_check = res.find('Featured Fact-check')
    # final_result += str(cnt) + '. ' + res[1:upto_fact_check]
    final_result += res[1:upto_fact_check] + " "
    cnt += 1

  return final_result

In [49]:
print(get_top_5_news_content(test_data['topics'][100]))

['Biden', "Trump's policies", 'Obama-Biden kids in cages policy', 'human trafficking', 'cartels']
With border crossings from Mexico rising sharply, fueled by a surge in unaccompanied children, former President Donald Trump painted a picture of doom in an interview with Fox News talk show host Sean Hannity.
Hannity asked about the rising number of child migrants and "Biden’s cages," a reference to overcrowded, soft-sided shelters where children are held temporarily. Trump responded more generally, saying his own policies left things well enough at the border, and all Biden had to do "was leave it alone." (We rated a similar claim Mostly False.)
Then he made a claim we wanted to check. 
"You know, the other thing that people don’t talk about — human trafficking and drugs," Trump said April 19. "That’s doubled, tripled and quadrupled coming in, because that’s pouring in right now."
The claim was contained in an excerpt of Trump’s interview shared on Facebook by his son Don Trump Jr., who 

In [46]:
print(topic_list)
system_content = """
You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the text provided. Ignore any bias mentioned in the news.
Summarize the below news articles in at most 100 words.
"""
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": system_content
    },
    {
      "role": "user",
      "content": get_top_5_news_content(topic_list)
    }
  ],
  temperature=0.3,
  max_tokens=256,
  top_p=0.3,
  frequency_penalty=1,
  presence_penalty=1
)

print(response["choices"][0]["message"]["content"].strip())

response = response["choices"][0]["message"]["content"].strip()
len(response.split(' '))

['Biden', "Trump's policies", 'Obama-Biden kids in cages policy', 'human trafficking', 'cartels']
Former President Donald Trump claimed in an interview that human trafficking and drug smuggling at the US-Mexico border had doubled, tripled, and quadrupled under President Joe Biden. However, there is no evidence to support this claim. Law enforcement agencies have not reported increases in trafficking victims or surges since Biden took office. Additionally, Trump falsely claimed during a debate that the Obama administration built cages for migrants at the border. While the Obama administration did hold immigrants in chain-link enclosures described as "cages," the photo shared on Facebook was from 2019 during Trump's presidency.


In [50]:
test_data['top_5_news'] = test_data.apply(lambda x: get_top_5_news_content(x['topics']), axis = 1)
test_data.head()

['Dan Patrick', 'drive-in voting', 'people of color', 'statistics', 'PolitiFact ruling']
['Biden speech', 'unvaccinated', 'lepers', 'missing relative at the dinner table', 'plagiarized.']
['face covering', 'concealed weapon']
['[formal challenge', "Georgia's electoral votes", 'Kelly Loeffler', 'House Republicans', 'riots]']
['Stand Up Fight Back Atlanta rally', 'anti-Asian violence', 'spa shooting victims', 'China']
['Trump', 'poverty rate', 'Wisconsin families', 'lowest rate in 22 years', 'unemployment rate', 'historic low', 'economy', 'USA chant']
['Trump', 'middle class', 'corporate cut', 'deceit and lies', 'Terrible Ted Cruz']
['VP of Academic Affairs', 'nursing home', 'HR friend in Florida']
['Ronald Reagan', 'amnesty', 'savings and loan industry collapse', 'taxes', 'arms sales to Iran', 'automatic weapons ban', 'open carry ban in CA', 'national debt']
['French Constitution', 'US Constitution', 'enumeration of rights', 'individual rights', 'subordination to the State']
['[Burisma'

,statement,cleaned_tweet,Truthfulness,topics,top_5_news
0,Statistics show thatmore people of color don't...,Dan PatrickIf theyre worried about people of c...,False,"Dan Patrick, drive-in voting, people of color,...",Eliminating drive-thru voting centers is one o...
1,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re...",Clips of President-elect Joe Biden’s speech du...
2,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon",A popular meme of Kevin Hart and Keanu Reeves ...
3,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ...",In a Georgia debate that could help decide whi...
4,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ...",Questions are swirling about Robert Aaron Long...


In [51]:
top_5_news_list = list(test_data['top_5_news'])
print(len(top_5_news_list))

104


In [52]:
import asyncio
import aiohttp
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + openai.api_key
}

system_content = """
You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the text provided. Ignore any bias mentioned in the news.
Summarize the below news articles in at most 100 words.
"""
class ProgressLog:
    def __init__(self, total):
        self.total = total
        self.done = 0

    def increment(self):
        self.done = self.done + 1

    def __repr__(self):
        return f"Done runs {self.done}/{self.total}."

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(20), before_sleep=print, retry_error_callback=lambda _: None)
async def get_completion(content, session, semaphore, progress_log):
    async with semaphore:

        async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json={
            "model": "gpt-3.5-turbo",
            "messages": [
                {
                  "role": "system",
                  "content": system_content
                },
                {
                  "role": "user",
                  "content": content
                }
              ],
            "temperature": 0.1,
            "max_tokens": 256,
            "top_p": 0.3,
            "frequency_penalty": 1.0,
            "presence_penalty": 1.0
        }) as resp:

            response_json = await resp.json()

            progress_log.increment()
            print(progress_log)

            return response_json["choices"][0]['message']["content"]

async def get_completion_list(content_list, max_parallel_calls, timeout):
    semaphore = asyncio.Semaphore(value=max_parallel_calls)
    progress_log = ProgressLog(len(content_list))

    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(timeout)) as session:
        return await asyncio.gather(*[get_completion(content, session, semaphore, progress_log) for content in content_list])

In [53]:
max_parallel_calls = 20
timeout = 200
generated_topics = await get_completion_list(top_5_news_list, max_parallel_calls, timeout)

Done runs 1/104.
<RetryCallState 140418759422208: attempt #1; slept for 0.78; last result: failed (KeyError 'choices')>
Done runs 2/104.
<RetryCallState 140418759324240: attempt #1; slept for 0.88; last result: failed (KeyError 'choices')>
Done runs 3/104.
<RetryCallState 140418759324624: attempt #1; slept for 0.42; last result: failed (KeyError 'choices')>
Done runs 4/104.
<RetryCallState 140418759321312: attempt #1; slept for 0.07; last result: failed (KeyError 'choices')>
Done runs 5/104.
Done runs 6/104.
Done runs 7/104.
<RetryCallState 140418759327504: attempt #1; slept for 0.24; last result: failed (KeyError 'choices')>
Done runs 8/104.
<RetryCallState 140418759322224: attempt #1; slept for 0.98; last result: failed (KeyError 'choices')>
Done runs 9/104.
<RetryCallState 140418759314304: attempt #1; slept for 0.96; last result: failed (KeyError 'choices')>
Done runs 10/104.
<RetryCallState 140418759313104: attempt #1; slept for 0.94; last result: failed (KeyError 'choices')>
Done 

In [54]:
len(generated_topics)

104

In [55]:
test_data['top_5_news_summary'] = generated_topics
test_data.head()

,statement,cleaned_tweet,Truthfulness,topics,top_5_news,top_5_news_summary
0,Statistics show thatmore people of color don't...,Dan PatrickIf theyre worried about people of c...,False,"Dan Patrick, drive-in voting, people of color,...",Eliminating drive-thru voting centers is one o...,None
1,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re...",Clips of President-elect Joe Biden’s speech du...,Facebook posts are circulating alleging that P...
2,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon",A popular meme of Kevin Hart and Keanu Reeves ...,A meme featuring Kevin Hart and Keanu Reeves i...
3,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ...",In a Georgia debate that could help decide whi...,"In a Georgia Senate debate, Democratic challen..."
4,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ...",Questions are swirling about Robert Aaron Long...,1. Speculation that the Atlanta spa shootings ...


In [58]:
test_data = test_data.dropna().reset_index()

TypeError: ignored

In [59]:
test_data.shape

(94, 7)

In [60]:
test_data['Truthfulness'].value_counts()

True     48
False    46
Name: Truthfulness, dtype: int64

In [61]:
test_data.to_csv("top_5_news_summarized_test_data.csv", index = False)

# Generate CoT for Truthfulness



In [62]:
test_data = pd.read_csv("top_5_news_summarized_test_data.csv")
test_data.head()

,index,statement,cleaned_tweet,Truthfulness,topics,top_5_news,top_5_news_summary
0,1,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re...",Clips of President-elect Joe Biden’s speech du...,Facebook posts are circulating alleging that P...
1,2,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon",A popular meme of Kevin Hart and Keanu Reeves ...,A meme featuring Kevin Hart and Keanu Reeves i...
2,3,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ...",In a Georgia debate that could help decide whi...,"In a Georgia Senate debate, Democratic challen..."
3,4,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ...",Questions are swirling about Robert Aaron Long...,1. Speculation that the Atlanta spa shootings ...
4,5,"""The unemployment rate for Wisconsin workers h...",Trump the poverty rate for Wisconsin families ...,True,"Trump, poverty rate, Wisconsin families, lowes...","At a April 27, 2019 rally in Green Bay before ...",1. President Trump claimed at a rally in Green...


In [64]:
test_data = test_data.drop('index', axis = 1)
test_data.head()

,statement,cleaned_tweet,Truthfulness,topics,top_5_news,top_5_news_summary
0,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re...",Clips of President-elect Joe Biden’s speech du...,Facebook posts are circulating alleging that P...
1,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon",A popular meme of Kevin Hart and Keanu Reeves ...,A meme featuring Kevin Hart and Keanu Reeves i...
2,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ...",In a Georgia debate that could help decide whi...,"In a Georgia Senate debate, Democratic challen..."
3,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ...",Questions are swirling about Robert Aaron Long...,1. Speculation that the Atlanta spa shootings ...
4,"""The unemployment rate for Wisconsin workers h...",Trump the poverty rate for Wisconsin families ...,True,"Trump, poverty rate, Wisconsin families, lowes...","At a April 27, 2019 rally in Green Bay before ...",1. President Trump claimed at a rally in Green...


In [91]:
text = test_data['cleaned_tweet'][27]
news_summary = test_data['top_5_news_summary'][27]
test_data['Truthfulness'][27]

True

In [94]:
system_content = """
You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the Text and News Summary provided.
Ignore any bias mentioned in the news. You will be provided with a Text and a News Summary. You need to identify if a Text is truthful to the News Summary.
Explain step by step why you think the Text is truthful and faithful to the News Summary or the Text is not truthful or faithful to the News Summary.
"""
print(system_content)

content = """
Text: {}
News Summary: {}
""".format(text, news_summary)
print(content)

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": system_content
    },
    {
      "role": "user",
      "content": content
    }
  ],
  temperature=0,
  max_tokens=256,
  top_p=0.3,
  frequency_penalty=0.5,
  presence_penalty=0.5
)

print(response["choices"][0]["message"]["content"].strip())


You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the Text and News Summary provided.
Ignore any bias mentioned in the news. You will be provided with a Text and a News Summary. You need to identify if a Text is truthful to the News Summary.
Explain step by step why you think the Text is truthful and faithful to the News Summary or the Text is not truthful or faithful to the News Summary.


Text: ur allowed to legally buy hemp but u cant grow it shit u can even make ur own alcohol if u want
News Summary: Lawyer William Devereaux spoke on behalf of the Narragansett Indian Tribe at a senate judiciary hearing, arguing for the legalization of hemp growth in Rhode Island. Currently, hemp is classified as marijuana under federal law and its cultivation is banned. However, historically, hemp has been used by Native American tribes for various products. The debate surrounding the leg

In [93]:
content_list = list()
for tdx, row in test_data.iterrows():
  # print(row['cleaned_tweet'], row['top_5_news_summary'])
  content = """
  Text: {}
  News Summary: {}
  """.format(row['cleaned_tweet'], row['top_5_news_summary'])
  content_list.append(content)

print(len(content_list))

94


In [96]:
import asyncio
import aiohttp
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + openai.api_key
}

system_content = """
You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the Text and News Summary provided.
Ignore any bias mentioned in the news. You will be provided with a Text and a News Summary. You need to identify if a Text is truthful to the News Summary.
Explain step by step why you think the Text is truthful and faithful to the News Summary or the Text is not truthful or faithful to the News Summary.
"""
class ProgressLog:
    def __init__(self, total):
        self.total = total
        self.done = 0

    def increment(self):
        self.done = self.done + 1

    def __repr__(self):
        return f"Done runs {self.done}/{self.total}."

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(20), before_sleep=print, retry_error_callback=lambda _: None)
async def get_completion(content, session, semaphore, progress_log):
    async with semaphore:

        async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json={
            "model": "gpt-3.5-turbo",
            "messages": [
                {
                  "role": "system",
                  "content": system_content
                },
                {
                  "role": "user",
                  "content": content
                }
              ],
            "temperature": 0.1,
            "max_tokens": 256,
            "top_p": 0.3,
            "frequency_penalty": 0.5,
            "presence_penalty": 0.5
        }) as resp:

            response_json = await resp.json()

            progress_log.increment()
            print(progress_log)

            return response_json["choices"][0]['message']["content"]

async def get_completion_list(content_list, max_parallel_calls, timeout):
    semaphore = asyncio.Semaphore(value=max_parallel_calls)
    progress_log = ProgressLog(len(content_list))

    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(timeout)) as session:
        return await asyncio.gather(*[get_completion(content, session, semaphore, progress_log) for content in content_list])

In [97]:
max_parallel_calls = 20
timeout = 200
generated_cot = await get_completion_list(content_list, max_parallel_calls, timeout)

Done runs 1/94.
Done runs 2/94.
Done runs 3/94.
Done runs 4/94.
Done runs 5/94.
Done runs 6/94.
Done runs 7/94.
Done runs 8/94.
Done runs 9/94.
Done runs 10/94.
Done runs 11/94.
Done runs 12/94.
Done runs 13/94.
Done runs 14/94.
Done runs 15/94.
Done runs 16/94.
Done runs 17/94.
Done runs 18/94.
Done runs 19/94.
Done runs 20/94.
Done runs 21/94.
Done runs 22/94.
Done runs 23/94.
Done runs 24/94.
Done runs 25/94.
Done runs 26/94.
Done runs 27/94.
Done runs 28/94.
Done runs 29/94.
Done runs 30/94.
Done runs 31/94.
Done runs 32/94.
Done runs 33/94.
Done runs 34/94.
Done runs 35/94.
Done runs 36/94.
Done runs 37/94.
Done runs 38/94.
Done runs 39/94.
Done runs 40/94.
Done runs 41/94.
Done runs 42/94.
Done runs 43/94.
Done runs 44/94.
Done runs 45/94.
Done runs 46/94.
Done runs 47/94.
Done runs 48/94.
Done runs 49/94.
Done runs 50/94.
Done runs 51/94.
Done runs 52/94.
Done runs 53/94.
Done runs 54/94.
Done runs 55/94.
Done runs 56/94.
Done runs 57/94.
Done runs 58/94.
Done runs 59/94.
Done r

In [99]:
len(generated_cot)

94

In [101]:
test_data['generated_CoT'] = generated_cot
test_data.head()

,statement,cleaned_tweet,Truthfulness,topics,top_5_news,top_5_news_summary,generated_CoT
0,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re...",Clips of President-elect Joe Biden’s speech du...,Facebook posts are circulating alleging that P...,The Text is not truthful and faithful to the N...
1,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon",A popular meme of Kevin Hart and Keanu Reeves ...,A meme featuring Kevin Hart and Keanu Reeves i...,The Text is not truthful and faithful to the N...
2,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ...",In a Georgia debate that could help decide whi...,"In a Georgia Senate debate, Democratic challen...",The Text is truthful and faithful to the News ...
3,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ...",Questions are swirling about Robert Aaron Long...,1. Speculation that the Atlanta spa shootings ...,The Text is not truthful and faithful to the N...
4,"""The unemployment rate for Wisconsin workers h...",Trump the poverty rate for Wisconsin families ...,True,"Trump, poverty rate, Wisconsin families, lowes...","At a April 27, 2019 rally in Green Bay before ...",1. President Trump claimed at a rally in Green...,"Based on the given Text and News Summary, it i..."


In [113]:
idx = 93
text = test_data['cleaned_tweet'][idx]
news_summary = test_data['top_5_news_summary'][idx]
cot = test_data['generated_CoT'][idx]
test_data['Truthfulness'][idx]

True

In [114]:
system_content = """
You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the Text and News Summary provided.
Ignore any bias mentioned in the news. You will be provided with a Text, News Summary, and Reasoning why the Text is truthful and faithful to the News Summary or the Text is not truthful or faithful to the News Summary.
Answer 'True' if the Text is truthful and faithful to the News Summary and 'False' if the Text is not truthful or faithful to the News Summary by analyzing the Reasoning in the context of the Text and News Summary
Just answer True or False.
"""
print(system_content)

content = """
Text: {}
News Summary: {}
Reasoning: {}
""".format(text, news_summary, cot)
print(content)

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": system_content
    },
    {
      "role": "user",
      "content": content
    }
  ],
  temperature=0,
  max_tokens=256,
  top_p=0.3,
  frequency_penalty=0.5,
  presence_penalty=0.5
)

print(response["choices"][0]["message"]["content"].strip())


You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the Text and News Summary provided.
Ignore any bias mentioned in the news. You will be provided with a Text, News Summary, and Reasoning why the Text is truthful and faithful to the News Summary or the Text is not truthful or faithful to the News Summary.
Answer 'True' if the Text is truthful and faithful to the News Summary and 'False' if the Text is not truthful or faithful to the News Summary by analyzing the Reasoning in the context of the Text and News Summary
Just answer True or False.


Text: US President Donald Trump said that increased COVID19 testing makes the US look bad by increasing coronavirus case numbers  I wish the press would ask him what more deaths make us look like PLEASE
News Summary: Donald Trump Jr. falsely claimed that the number of coronavirus deaths had plummeted to "almost nothing," dismissing the i

In [119]:
content_list = list()
for tdx, row in test_data.iterrows():
  # print(row['cleaned_tweet'], row['top_5_news_summary'])
  content = """
  Text: {}
  News Summary: {}
  Reasoning: {}
  """.format(row['cleaned_tweet'], row['top_5_news_summary'], row['generated_CoT'])
  content_list.append(content)

print(len(content_list))
print(content_list[0])

94

  Text: Im literally laughing out loud at the biden speech keeps The unvaccinated like they are lepers The line about a missing relative at the dinner table He copied his own line from the debate with trump at least he plagiarized himself this time this sounds like an SNL skit
  News Summary: Facebook posts are circulating alleging that President Joe Biden plagiarized speeches from former President Donald Trump and made false claims about United Airlines' COVID-19 vaccine mandate. However, there is no evidence to support these allegations. Another post falsely claimed that Supreme Court Justice Sonia Sotomayor was seen dining with Democratic leaders when it was actually Schumer's wife. Additionally, a screenshot of a headline claiming Biden ordered the withholding of health benefits from unvaccinated veterans is fake news from a satirical website.
  Reasoning: The Text is not truthful and faithful to the News Summary. Here's why:

1. The Text mentions that Biden plagiarized speeche

In [117]:
import asyncio
import aiohttp
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + openai.api_key
}

system_content = """
You are an expert journalist. You do not hallucinate. Your views are not aligned to any political party or propaganda. You are unbiased, neutral and faithful to the Text and News Summary provided.
Ignore any bias mentioned in the news. You will be provided with a Text, News Summary, and Reasoning why the Text is truthful and faithful to the News Summary or the Text is not truthful or faithful to the News Summary.
Answer 'True' if the Text is truthful and faithful to the News Summary and 'False' if the Text is not truthful or faithful to the News Summary by analyzing the Reasoning in the context of the Text and News Summary
Just answer True or False.
"""
class ProgressLog:
    def __init__(self, total):
        self.total = total
        self.done = 0

    def increment(self):
        self.done = self.done + 1

    def __repr__(self):
        return f"Done runs {self.done}/{self.total}."

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(20), before_sleep=print, retry_error_callback=lambda _: None)
async def get_completion(content, session, semaphore, progress_log):
    async with semaphore:

        async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json={
            "model": "gpt-3.5-turbo",
            "messages": [
                {
                  "role": "system",
                  "content": system_content
                },
                {
                  "role": "user",
                  "content": content
                }
              ],
            "temperature": 0.1,
            "max_tokens": 256,
            "top_p": 0.3,
            "frequency_penalty": 0.5,
            "presence_penalty": 0.5
        }) as resp:

            response_json = await resp.json()

            progress_log.increment()
            print(progress_log)

            return response_json["choices"][0]['message']["content"]

async def get_completion_list(content_list, max_parallel_calls, timeout):
    semaphore = asyncio.Semaphore(value=max_parallel_calls)
    progress_log = ProgressLog(len(content_list))

    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(timeout)) as session:
        return await asyncio.gather(*[get_completion(content, session, semaphore, progress_log) for content in content_list])

In [120]:
max_parallel_calls = 20
timeout = 200
predicted_truthfulness = await get_completion_list(content_list, max_parallel_calls, timeout)

Done runs 1/94.
Done runs 2/94.
Done runs 3/94.
Done runs 4/94.
Done runs 5/94.
Done runs 6/94.
Done runs 7/94.
Done runs 8/94.
Done runs 9/94.
Done runs 10/94.
Done runs 11/94.
Done runs 12/94.
Done runs 13/94.
Done runs 14/94.
Done runs 15/94.
Done runs 16/94.
Done runs 17/94.
Done runs 18/94.
Done runs 19/94.
Done runs 20/94.
Done runs 21/94.
Done runs 22/94.
Done runs 23/94.
Done runs 24/94.
Done runs 25/94.
Done runs 26/94.
Done runs 27/94.
Done runs 28/94.
Done runs 29/94.
Done runs 30/94.
Done runs 31/94.
Done runs 32/94.
Done runs 33/94.
Done runs 34/94.
Done runs 35/94.
Done runs 36/94.
Done runs 37/94.
Done runs 38/94.
Done runs 39/94.
Done runs 40/94.
Done runs 41/94.
Done runs 42/94.
Done runs 43/94.
Done runs 44/94.
Done runs 45/94.
Done runs 46/94.
Done runs 47/94.
Done runs 48/94.
Done runs 49/94.
Done runs 50/94.
Done runs 51/94.
Done runs 52/94.
Done runs 53/94.
Done runs 54/94.
Done runs 55/94.
Done runs 56/94.
Done runs 57/94.
Done runs 58/94.
Done runs 59/94.
Done r

In [121]:
test_data['predicted_truthfulness'] = predicted_truthfulness
test_data.head()

,statement,cleaned_tweet,Truthfulness,topics,top_5_news,top_5_news_summary,generated_CoT,predicted_truthfulness
0,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re...",Clips of President-elect Joe Biden’s speech du...,Facebook posts are circulating alleging that P...,The Text is not truthful and faithful to the N...,False
1,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon",A popular meme of Kevin Hart and Keanu Reeves ...,A meme featuring Kevin Hart and Keanu Reeves i...,The Text is not truthful and faithful to the N...,False
2,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ...",In a Georgia debate that could help decide whi...,"In a Georgia Senate debate, Democratic challen...",The Text is truthful and faithful to the News ...,False
3,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ...",Questions are swirling about Robert Aaron Long...,1. Speculation that the Atlanta spa shootings ...,The Text is not truthful and faithful to the N...,False
4,"""The unemployment rate for Wisconsin workers h...",Trump the poverty rate for Wisconsin families ...,True,"Trump, poverty rate, Wisconsin families, lowes...","At a April 27, 2019 rally in Green Bay before ...",1. President Trump claimed at a rally in Green...,"Based on the given Text and News Summary, it i...",False


In [129]:
print(type(test_data['Truthfulness'][0]))

<class 'numpy.bool_'>


In [131]:
cnt = 0
for idx, row in test_data.iterrows():
  if str(row['Truthfulness']) == str(row['predicted_truthfulness'].strip()):
    cnt += 1
print("Accuracy: ", cnt/test_data.shape[0])

Accuracy:  0.5957446808510638


In [135]:
test_data.head()

,statement,cleaned_tweet,Truthfulness,topics,top_5_news,top_5_news_summary,generated_CoT,predicted_truthfulness
0,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re...",Clips of President-elect Joe Biden’s speech du...,Facebook posts are circulating alleging that P...,The Text is not truthful and faithful to the N...,False
1,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon",A popular meme of Kevin Hart and Keanu Reeves ...,A meme featuring Kevin Hart and Keanu Reeves i...,The Text is not truthful and faithful to the N...,False
2,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ...",In a Georgia debate that could help decide whi...,"In a Georgia Senate debate, Democratic challen...",The Text is truthful and faithful to the News ...,False
3,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ...",Questions are swirling about Robert Aaron Long...,1. Speculation that the Atlanta spa shootings ...,The Text is not truthful and faithful to the N...,False
4,"""The unemployment rate for Wisconsin workers h...",Trump the poverty rate for Wisconsin families ...,True,"Trump, poverty rate, Wisconsin families, lowes...","At a April 27, 2019 rally in Green Bay before ...",1. President Trump claimed at a rally in Green...,"Based on the given Text and News Summary, it i...",False


In [136]:
test_data.to_csv("predicted_truthfulness_final_2_class.csv", index = False)

# Evaluation

In [6]:
df = pd.read_csv("predicted_truthfulness_final_2_class.csv")
df.head()

,statement,cleaned_tweet,Truthfulness,topics,top_5_news,top_5_news_summary,generated_CoT,predicted_truthfulness
0,Says Joe Biden plagiarized Donald Trump in his...,Im literally laughing out loud at the biden sp...,False,"Biden speech, unvaccinated, lepers, missing re...",Clips of President-elect Joe Biden’s speech du...,Facebook posts are circulating alleging that P...,The Text is not truthful and faithful to the N...,False
1,If you are licensed to carry a concealed weapo...,And that is why I havent worn one and will not...,False,"face covering, concealed weapon",A popular meme of Kevin Hart and Keanu Reeves ...,A meme featuring Kevin Hart and Keanu Reeves i...,The Text is not truthful and faithful to the N...,False
2,Photo shows Kelly Loefflers house.,NO formal challenge to Georgias electoral vote...,False,"[formal challenge, Georgia's electoral votes, ...",In a Georgia debate that could help decide whi...,"In a Georgia Senate debate, Democratic challen...",The Text is truthful and faithful to the News ...,False
3,The Atlanta shooting suspect wrote an anti-Chi...,Stand Up Fight Back Atlanta rally decries anti...,False,"Stand Up Fight Back Atlanta rally, anti-Asian ...",Questions are swirling about Robert Aaron Long...,1. Speculation that the Atlanta spa shootings ...,The Text is not truthful and faithful to the N...,False
4,"""The unemployment rate for Wisconsin workers h...",Trump the poverty rate for Wisconsin families ...,True,"Trump, poverty rate, Wisconsin families, lowes...","At a April 27, 2019 rally in Green Bay before ...",1. President Trump claimed at a rally in Green...,"Based on the given Text and News Summary, it i...",False


In [8]:
y_true = df['Truthfulness']
y_pred = df['predicted_truthfulness']

tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(y_true)):
  if str(y_true[i]) == str(y_pred[i]) == 'True':
    tp += 1
  elif str(y_true[i]) == str(y_pred[i]) == 'False':
    tn += 1
  elif str(y_true[i]) == 'True' and str(y_true[i] != str(y_pred[i])):
    fn += 1
  else:
    fp += 1
print(f"tp: {tp} \n tn:{tn} \n fp: {fp} \n fn: {fn}")

pr = (tp)/(tp + fp)
re = tp/(tp+fn)
print("Accuracy: ", (tp+tn)/(tp + tn + fp + fn))
print("Precision: ", (tp)/(tp + fp))
print("Recall: ", tp/(tp+fn))
print("F1: ", (2*pr*re)/(pr+re))

tp: 15 
 tn:41 
 fp: 5 
 fn: 33
Accuracy:  0.5957446808510638
Precision:  0.75
Recall:  0.3125
F1:  0.4411764705882353


# Hybrid Model

In [ ]:
df = pd.read_csv('')